In [1]:
"""

 similar_images_TL.py  (author: Anson Wong / git: ankonzoid)

 We find similar images in a database by using transfer learning
 via a pre-trained VGG image classifier. We plot the 5 most similar
 images for each image in the database, and plot the tSNE for all
 our image feature vectors.

"""

'\n\n similar_images_TL.py  (author: Anson Wong / git: ankonzoid)\n\n We find similar images in a database by using transfer learning\n via a pre-trained VGG image classifier. We plot the 5 most similar\n images for each image in the database, and plot the tSNE for all\n our image feature vectors.\n\n'

In [52]:
import sys, os
import numpy as np
from keras.preprocessing import image
from keras.models import Model
sys.path.append("src")

from vgg19 import VGG19
from imagenet_utils import preprocess_input
from plot_utils import plot_query_answer
from sort_utils import find_topk_unique
from kNN import kNN
from tSNE import plot_tsne

import shutil

In [53]:
# ================================================
# Load pre-trained model and remove higher level layers
# ================================================
print("Loading VGG19 pre-trained model...")
base_model = VGG19(weights='imagenet')
model = Model(input=base_model.input,
              output=base_model.get_layer('block4_pool').output)

Loading VGG19 pre-trained model...


/opt/rh/rh-python35/root/usr/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  import sys


In [54]:
# ================================================
# RAW DATA : Read images and convert them to feature vectors 
# ================================================
imgs, filename_heads, X, file_full_path = [], [], [], []
path = "charts"
print("Reading images from '{}' directory...\n".format(path))
for f in os.listdir(path):
    #print(f)
    # Process filename
    filename = os.path.splitext(f)  # filename in directory
    filename_full = os.path.join(path,f)  # full path filename
    file_full_path.append(filename_full)
    head, ext = filename[0], filename[1]
    if ext.lower() not in [".jpg", ".jpeg"]:
        continue

    # Read image file
    img = image.load_img(filename_full, target_size=(224, 224))  # load
    imgs.append(np.array(img))  # image List
    filename_heads.append(head)  # filename head list

    # Pre-process for model input
    img = image.img_to_array(img)  # convert to array
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = model.predict(img).flatten()  # features
    X.append(features)  # append feature extractor

Reading images from 'charts' directory...



/opt/rh/rh-python35/root/usr/lib64/python3.5/site-packages/PIL/Image.py:953: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [58]:
# ================================================
# QUERY IMAGES : Read images and convert them to feature vectors 
# ================================================

def query_images(path='test') :

    query_features,query_img = [], []
    print("Reading images from '{}' directory...\n".format(path))
    for f in os.listdir(path):
        #print(f)
        # Process filename
        filename = os.path.splitext(f)  # filename in directory
        filename_full = os.path.join(path,f)  # full path filename
        query_img.append(filename_full)
        head, ext = filename[0], filename[1]
        if ext.lower() not in [".jpg", ".jpeg"]:
            continue

        # Read image file
        img = image.load_img(filename_full, target_size=(224, 224))  # load
        #imgs.append(np.array(img))  # image List
        #filename_heads.append(head)  # filename head list

        # Pre-process for model input
        img = image.img_to_array(img)  # convert to array
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        features = model.predict(img).flatten()  # features
        query_features.append(features)  # append feature extractor

    query_features = np.array(query_features)
    
    return query_features, query_img

In [59]:
X = np.array(X)  # feature vectors
imgs = np.array(imgs)  # images
print("imgs.shape = {}".format(imgs.shape))
print("X_features.shape = {}\n".format(X.shape))

imgs.shape = (7926, 224, 224, 3)
X_features.shape = (7926, 100352)



In [30]:
# ===========================
# Find k-nearest images to each image
# ===========================
n_neighbours = 25 + 1  # +1 as itself is most similar
knn = kNN()  # kNN model
knn.compile(n_neighbors=n_neighbours, algorithm="brute", metric="cosine")
knn.fit(X)

In [31]:
query_features,query_img = query_images()
print("Query_features.shape = {}\n".format(query_features.shape))
nearest_test = 'output_test'

Reading images from 'test' directory...

Query_features.shape = (15, 100352)



In [32]:
#n_neighbours = 20
for index,img in enumerate(query_img) :
    
    distances, indices = knn.predict(np.array([query_features[index]]))
    distances = distances.flatten()
    indices = indices.flatten()
    indices, distances = find_topk_unique(indices, distances, n_neighbours)
    #print("Nearest Neighbours index : {} ".format(indices))
    

    for idx in  np.nditer(indices) :
        #print("File Name : {} ; Index : {}".format(query_img[idx],idx))
        shutil.copy(file_full_path[idx],nearest_test)
        #break
    #break

In [60]:
np.save('feature_vector',X)

In [61]:
np.save('data_imgs',file_full_path)

In [62]:
X.shape

(7926, 100352)